In [21]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

## Web scraping with Beautiful Soup

In [23]:
path='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
source=requests.get(path).text
soup=BeautifulSoup(source,'html.parser')

table=soup.find('table')
#print(table.prettify())
rows=table.find_all('tr')

data=[]
for row in rows:
    #print(row.prettify())
    fields=row.find_all('td')
    #print(fields)
    #print('-------------')
    if fields:
        data.append(fields)
#print(data)
postal_code=[]

df_list=[]
for item in data:
    #print(item)
    postal_code=item[0].text.strip('\n')
    borough=item[1].text.strip('\n')
    neighbourhood=item[2].text.strip('\n')
    if borough != 'Not assigned':
        if neighbourhood == 'Not assigned':
            neighborhood = borough
        df_list.append([postal_code, borough, neighbourhood])
#print(df)

## Converting the list to a dataframe

In [27]:
df=pd.DataFrame(df_list)
df.columns=['Postal_Code','Borough','Neighbourhood']
df

,Postal_Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor
...,...,...,...
205,M8Z,Etobicoke,Kingsway Park South West
206,M8Z,Etobicoke,Mimico NW
207,M8Z,Etobicoke,The Queensway West
208,M8Z,Etobicoke,Royal York South West


## Combining rows having same postal code but different neighbourhoods.

In [29]:
df = df.groupby('Postal_Code').agg(
    {
        'Borough':'first', 
        'Neighbourhood': ', '.join,}
    ).reset_index()
df

,Postal_Code,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv..."
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ..."


In [34]:
df.shape

(103, 3)